In [81]:
import sys
sys.path.append('../')
import pandas as pd
import random
import os
import numpy as np
from cnt.generate_re_data import AutoRE

from cnt.annotate import (annotate, annotate_single_design, 
                          annotate_designs, 
                          extract_string_from_annotation, labeling_eng)
from cnt.extract_relation import (path, NERTransformer, FeatureExtractor)
from cnt.evaluate import Metrics
from cnt.vectorize import (Doc2Str, Path2Str, Verbs2Str, AveragedPath2Vec, 
                           AveragedRest2Vec)
from cnt.io import (replace_left_right)
from cnt.io import  Database_Connection
from cnt.preprocess import Preprocess

import warnings
warnings.filterwarnings('ignore')

In [82]:
auto = AutoRE()

In [83]:
subjects = ["Athena", "Ares", "Apollo", "Zeus", "Nike", "Olybrius", "Seleucus I", "Domitianus", "Equitas", "Rhoemetalces I"]

In [84]:
relations = {
    "holding":["sword", "shield", "paludamentum", "staff", "club", "stones", "jar", "bow"],
    "wearing":["belt", "crown", "helmet", "cuirass", "crown juwel", "ribbons", "boots"],
    "seated_on":["dolphin", "shield", "omphalos", "rock", "bull", "panther", "kline"],
    "resting_on":["serpent staff", "torch", "shield", "panther", "club", "water urn", "knee", "rock"],
    "feeding":["panther", "dolphin"],
    "receiving":["apple", "agonistic crown", "crown", "shield"],}

In [85]:
auto.set_subjects(subjects)

In [86]:
auto.set_relations(relations)

In [87]:
result = auto.generate(300)

In [88]:
result.head(5)

In [89]:
X = result[["design_en"]]
X["design_de"] = ""
X["comment"] = "auto_generated"
X["design_en_changed"] = ""
X.head(2)

In [90]:
X.shape

In [91]:
dc =  Database_Connection("mysql+mysqlconnector://...") # Format user:password@IP/Database

In [92]:
X.to_sql("nlp_training_designs",dc.mysql_connection,if_exists="append", index=False)

In [93]:
def get_id(design):
    try:
        return dc.create_own_query("select id from nlp_training_designs where design_en='"+design+"';").id.item()
    except:
        return "Null"

In [94]:
X["db_id"] = X.apply(lambda row: get_id(row.design_en), axis=1)

In [95]:
X.head(5)

In [96]:
X = X.loc[X.db_id!="Null"]
X.shape

In [97]:
X["y"] = result["y"]

In [98]:
X.head(5)

## Add relations to db

In [99]:
entities = dc.create_own_query("select * from nlp_list_entities;")
entities[["id", "name_en", "alternativenames_en"]]

In [100]:
X["y"] = X.apply(lambda row: [row.y], axis=1)

In [101]:
ent_dict = {}
for index, row in entities[["id", "name_en", "alternativenames_en"]].iterrows():
    ent_dict[row["id"]] = (row["name_en"]+", "+row["alternativenames_en"]).replace(", None","").lower()

In [102]:
result = pd.DataFrame(columns=["design_id", "subject", "predicate", "object"])

In [103]:
def get_id(ent):
    for i in ent_dict:
        if ent.lower() in ent_dict[i]:
            return i
    return ent

In [104]:
for index, row in X.iterrows():
    for y in row.y:
        subject = get_id(y[0])
        subject_str = y[0]
        predicate = get_id(y[1])
        predicate_str = y[1]
        obj = get_id(y[2])
        obj_str = y[2]
        if type(subject) == int and type(predicate) == int and type(obj) == int:
            result = result._append({"design_id": row.db_id, 
                                     "subject": subject, 
                                     "subject_str": subject_str,
                                     "predicate": predicate, 
                                     "predicate_str": predicate_str,
                                     "object": obj,
                                     "object_str": obj_str}, ignore_index=True)
        else:
            print(subject, predicate, obj)

In [105]:
result.shape

In [ ]:
result.to_sql("nlp_relation_extraction_en_v2",dc.mysql_connection,if_exists="append", index=False)